In [182]:

import pkg_resources
import subprocess
import openai
import re
import json

def install_if_needed(package):
    try:
        dist = pkg_resources.get_distribution(package)
        print(f"{package} is already installed (version {dist.version}).")
    except pkg_resources.DistributionNotFound:
        print(f"{package} is not installed. Installing...")
        subprocess.call(['pip3', 'install', package])
        print(f"{package} has been installed.")
        
install_if_needed('openai')

openai is already installed (version 0.27.8).


In [183]:
#https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
#https://www.youtube.com/watch?v=uCKH8bmPgFs&t=502s

In [184]:
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = "https://gpt4resource.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "618bf90290544e54b47bcfc2dba743da"

In [185]:
language = 'Node.js'
topic = 'web development'
difficulty = 'intermediate'

In [253]:
system_message = 'Assistant is an intelligent chatbot designed to help users create programming training materials'
user_message = f"""Could you please provide me with incorrect code in {language}, followed by a corresponding unit test that will fail? 
                        Then provide corrected code that will pass the unit test. 
                        For each code snippet, provide an appropriate file name. 
                        The area of interest is {topic}. 
                        The difficulty should be {difficulty}."""

In [254]:
print(user_message)

Could you please provide me with incorrect code in Node.js, followed by a corresponding unit test that will fail? 
                        Then provide corrected code that will pass the unit test. 
                        For each code snippet, provide an appropriate file name. 
                        The area of interest is web development. The difficulty should be intermediate.


In [255]:
response = openai.ChatCompletion.create(
    #gpt-35-turbo max tokens = 4,096
    engine="testdeploy", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ],
    temperature = 0 # This is the degree of randomness of the model's output
)

In [256]:
# print(response)
# print(50*'-')

output = response['choices'][0]['message']['content']

print('tokens: ', response['usage'])
print(100*'-')
print(100*'-')
print(output)

tokens:  {
  "completion_tokens": 325,
  "prompt_tokens": 91,
  "total_tokens": 416
}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Sure, here's an example:

**Incorrect code:**

File name: `server.js`

```javascript
const http = require('http');

const server = http.createServer((req, res) => {
  res.statusCode = 200;
  res.setHeader('Content-Type', 'text/plain');
  res.write('Hello, World!');
  res.end();
});

server.listen(3000, () => {
  console.log('Server running on port 3000');
});
```

**Unit test that will fail:**

File name: `server.test.js`

```javascript
const assert = require('assert');
const http = require('http');

describe('Server', () => {
  it('should return "Hello, World!"', (done) => {
    http.get('http://localhost:3000', (res) => {
      assert.equal(res.statusCode, 200);
      assert.equal(res.headers['content

In [257]:
def extract_languages(text):
    pattern = r"```(.*)\n"
    matches = re.findall(pattern, text)
    languages = [match for match in matches if match != '']
    return languages

def extract_filenames(text):
    pattern = r"File name: `(.*)`\n"
    matches = re.findall(pattern, text)
    matches.pop()
    matches.append('corrected-' + matches[0].split('.')[0] + '.' + matches[0].split('.')[1])
    return matches

def remove_before_second_newline(text):
    lines = text.split('\n')
    new_text = '\n'.join(lines[2:])
    if new_text[0] == '\n':
        return new_text[1:]
    else:
        return new_text

def extract_code(text):
    pattern = r"```([\s\S]*?)```"
    matches = re.findall(pattern, text)
    code = []
    for match in matches:
        code.append(remove_before_second_newline(match))
    return code

In [258]:
languages = extract_languages(output)
filenames = extract_filenames(output)
code = extract_code(output)

In [259]:
languages

['javascript', 'javascript', 'javascript']

In [260]:
filenames

['server.js', 'server.test.js', 'corrected-server.js']

In [261]:
code

["const server = http.createServer((req, res) => {\n  res.statusCode = 200;\n  res.setHeader('Content-Type', 'text/plain');\n  res.write('Hello, World!');\n  res.end();\n});\n\nserver.listen(3000, () => {\n  console.log('Server running on port 3000');\n});\n",
 'const http = require(\'http\');\n\ndescribe(\'Server\', () => {\n  it(\'should return "Hello, World!"\', (done) => {\n    http.get(\'http://localhost:3000\', (res) => {\n      assert.equal(res.statusCode, 200);\n      assert.equal(res.headers[\'content-type\'], \'text/plain\');\n      res.on(\'data\', (data) => {\n        assert.equal(data.toString(), \'Hello, Node!\');\n      });\n      done();\n    });\n  });\n});\n',
 "const server = http.createServer((req, res) => {\n  res.statusCode = 200;\n  res.setHeader('Content-Type', 'text/plain');\n  res.write('Hello, Node!');\n  res.end();\n});\n\nserver.listen(3000, () => {\n  console.log('Server running on port 3000');\n});\n"]

In [262]:
data = {
    'labels': ['Incorrect code', 'Unit test', 'Corrected code'],
    'languages': languages,
    'filenames': filenames,
    'code': code
}

json_filename = 'label_lang_file_code.json'
with open(json_filename, 'w') as file:
    json.dump(data, file)

In [28]:
#RUN CELL TO SAVE OUTPUT IN A TXT FILE
import os

folder_path = 'txt_outputs'
text_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
txt_number_list = []
for file_name in text_files:
    txt_number = file_name[-5]
    txt_number = int(txt_number)
    txt_number_list.append(txt_number)
if not txt_number_list: #check if list is empty, i.e. no txt files yet
    new_txt_number = str(1)
else:
    new_txt_number = str(max(txt_number_list) + 1)
text_file = open('txt_outputs/output' + '_' + new_txt_number + '.txt', 'w')
text_file.write(system_message + '\n')
text_file.write(50*'-' + '\n')
text_file.write(user_message + '\n')
text_file.write(50*'-' + '\n')
text_file.write(output + '\n')
text_file.close()